In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
from datetime import date
plt.style.use('ggplot')

In [2]:
districts = list(pd.read_csv('districts.csv', header=None).iloc[:,0])
poll_df = pd.read_excel('PoliStat polls (Responses).xlsx', sheet='Form Responses 1')

print(poll_df.columns)

Index(['Timestamp', 'District Name (e.g. AZ-03, WA-11)', 'Final date taken',
       'Democratic percent (Not margin, not 2-party)', 'Sample Size',
       'Voter model',
       'House Effects (538 correction factor, + if blue arrow, - if red arrow)',
       'Pollster Grade https://projects.fivethirtyeight.com/pollster-ratings/ (Put D if N/A)',
       'Republican percent (Not margin, not 2-party)', 'Pollster Name'],
      dtype='object')


In [11]:
WEIGHTS = {'A': 1,
           'B': 2,
           'C': 3,
           'D': 4}

polls = {district: [] for district in districts}
for index, row in poll_df.iterrows():
    pv = row['Democratic percent (Not margin, not 2-party)']
    grade = WEIGHTS[row['Pollster Grade https://projects.fivethirtyeight.com/pollster-ratings/ (Put D if N/A)']]
    
    year = int(str(row['Final date taken']).split(' ')[0].split('-')[0])
    month = int(str(row['Final date taken']).split(' ')[0].split('-')[1])
    day = int(str(row['Final date taken']).split(' ')[0].split('-')[2])
    d0 = date(year, month, day)
    d1 = date(2018, 11, 6)
    days = d1 - d0
    days = days.days
    
    ssize = row['Sample Size']
    
    polls[row['District Name (e.g. AZ-03, WA-11)']].append([pv, grade, days, ssize])
    
maxlen = 0
for poll in polls:
    if len(polls[poll]) > maxlen:
        maxlen = len(polls[poll])



maxlen: 9


In [7]:
# Prior to Python 3.6, dicts do not preserve order and this output will not preserve district order
# Run this thing in 3.6+ (the way this notebook is currently configured to run) or suffer.
for poll in polls:
    if len(polls[poll]) > 0:
        rep = np.array(polls[poll])
        polls[poll] = np.average(rep[:,0], weights=rep[:,1])
    else:
        polls[poll] = 999
with open('ppoll.csv', 'w') as f:
    [f.write('{0},{1}\n'.format(key, value)) for key, value in polls.items()]

In [10]:
lhm = pd.read_excel('l_handmod.xlsx', sheet='Sheet 1', header=None)
lhm.columns = ['Name'] + ['S' + str(rep).zfill(3) for rep in range(lhm.shape[1] - 1)]

In [ ]:
def train(dset):
    